In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, mannwhitneyu 
import seaborn as sns 
from mpl_toolkits.mplot3d import Axes3D
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
%matplotlib inline

In [3]:
df = pd.read_csv('Desktop/Thinkful/states_all.csv')
df.head(2)

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 26 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1492 non-null float64
TOTAL_REVENUE                   1492 non-null float64
FEDERAL_REVENUE                 1492 non-null float64
STATE_REVENUE                   1492 non-null float64
LOCAL_REVENUE                   1492 non-null float64
TOTAL_EXPENDITURE               1492 non-null float64
INSTRUCTION_EXPENDITURE         1492 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1492 non-null float64
OTHER_EXPENDITURE               1492 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1492 non-null float64
GRADES_PK_G                     1492 non-null float64
GRADES_KG_G                     1492 non-null float64
GRADES_4_G                      1492 non-null float64
GRADES_8_G                      1

In [4]:
# Create a list of all columns we want to work on.
fill_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE", 
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE", 
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G", 
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G", 
             "GRADES_9_12_G", "GRADES_ALL_G"]
# Fill in all missing values from the column with the average of the same column
states = df["STATE"].unique()

for col in fill_list:
    for state in states: 
        if len(df.loc[df["STATE"] == state][col].unique())==1: 
            #col value is all null for that year, then fill it with 0 
            df.loc[df["STATE"] == state,  col]= df.loc[df["STATE"] == state,  col].fillna(0)
        else: 
            df.loc[df["STATE"] == state,  col] = df.loc[df["STATE"] == state, col].fillna(df[(df["STATE"] == state) & (df[col].isnull()==False) ][col].mean())

Say, we want to understand the relationship between the expenditures of the governments and the students' overall success in the math and reading.

## 1. Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.


In [5]:
df["score"] = (df["GRADES_4_G"]*((df["AVG_MATH_4_SCORE"] + df["AVG_READING_4_SCORE"])*0.5) + df["GRADES_8_G"]
                                 * ((df["AVG_MATH_8_SCORE"] + df["AVG_READING_8_SCORE"])*0.5))/(df["GRADES_4_G"] + df["GRADES_8_G"])


We weighted the score variables using the number of students in the respective grades.

## 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?


In [11]:
df[["score", 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
score,1.000000,0.082445,0.095358,0.087077,0.008600,0.005121
TOTAL_EXPENDITURE,0.082445,1.000000,0.991905,0.993717,0.942618,0.936786
INSTRUCTION_EXPENDITURE,0.095358,0.991905,1.000000,0.978307,0.908899,0.902719
SUPPORT_SERVICES_EXPENDITURE,0.087077,0.993717,0.978307,1.000000,0.949653,0.923064
OTHER_EXPENDITURE,0.008600,0.942618,0.908899,0.949653,1.000000,0.918804
CAPITAL_OUTLAY_EXPENDITURE,0.005121,0.936786,0.902719,0.923064,0.918804,1.000000


The most correlated variable is SUPPORT_SERVICES_EXPENDITURE

## 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?


In [12]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [16]:
X = df[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)
sklearn_pca = PCA(n_components=1)
df["pca_1"] = sklearn_pca.fit_transform(X)
print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.94778051]


More than 94% of the total variance is explained by the first principal component.

## 4. What is the correlation between the overall score variable and the 1st principal component?


In [18]:
df[['score', "pca_1"]].corr()

,score,pca_1
score,1.000000,0.050642
pca_1,0.050642,1.000000


## 5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

In [19]:
df[['score', "pca_1", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,score,pca_1,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
score,1.000000,0.050642,0.095358,0.087077,0.008600,0.005121
pca_1,0.050642,1.000000,0.973487,0.989163,0.970006,0.961295
INSTRUCTION_EXPENDITURE,0.095358,0.973487,1.000000,0.978307,0.908899,0.902719
SUPPORT_SERVICES_EXPENDITURE,0.087077,0.989163,0.978307,1.000000,0.949653,0.923064
OTHER_EXPENDITURE,0.008600,0.970006,0.908899,0.949653,1.000000,0.918804
CAPITAL_OUTLAY_EXPENDITURE,0.005121,0.961295,0.902719,0.923064,0.918804,1.000000


Instruction expenditure variable is more correlated with the overall score than the first principal component. Hence using instruction expenditure makes more sense. PCA works best when the correlation between the variables are less than and equal to 0.8. In our case, all of the expenditure variables are highly correlated with each other. This may result in instable principal component estimations.